In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [2]:
from case_code import CodeDownloader, get_metadata, CodeLoader, transform, Case
from utils.model_selector import get_chat_model

In [3]:
# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()
print(f"{'Total files downloaded:':<24} {downloaded_count}")

metadata = get_metadata()
print(f"{'Total cases in metadata:':<24} {len(metadata)}")

Total files downloaded:  125
Total cases in metadata: 110


In [4]:
import pandas as pd

# Get the chat model
model = get_chat_model()

# Transform the downloaded code to case
loader = CodeLoader()
skipped = await transform(model=model, loader=loader, total=downloaded_count)

data = [(key.value, len(value)) for key, value in skipped.items()]
df = pd.DataFrame(data, columns=["Error", "Count"])
print(df)

Processing: 100%|██████████| 125/125 [03:52<00:00,  1.86s/it]

            Error  Count
0  Case Not Found     57
1     Parse Error      0


In [5]:
from case_code.code_transformer import TransformError

print(f"Parsed error cases: {skipped[TransformError.ParseError]}")

Parsed error cases: []


In [6]:
from case import get_retriever

retriever = get_retriever()

In [7]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 2, 'source': 'eigenpie/index.ts', 'steps': "[{'description': 'Stake ETH to Lido', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': 'unknown', 'function_name': 'submit', 'input_args': ['unknown']}, {'description': 'Approve stETH to Eigenpie', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '0', 'function_name': 'approve', 'input_args': ['0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'unknown']}, {'description': 'Stake stETH to Eigenpie', 'to': '0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'value': '0', 'function_name': 'depositAsset', 'input_args': ['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'unknown', 'unknown', 'unknown']}]", 'total_steps': 3}, page_content='A case for staking ETH to Lido and then to Eigenpie using the Eigenpie Staking contract.'),
 Document(metadata={'case_id': 'zircuit_eigenpie_swell', 'seq_num': 48, 'source': 'zircuit-eigenpie-swell/index.ts', 'steps': "[{'description': 'Stake ET

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [9]:
question = "Stake 0.01 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "eigenpie_steth", "description": "Stake 0.01 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0.01", "function_name": "submit", "input_args": ["0.01"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "unknown"]}, {"description": "Stake stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "unknown", "unknown", "unknown"]}]}


In [10]:
question = "Stake 3 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_3_eth_lido_deposit_eigenpie", "description": "Stake 3 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "3", "function_name": "submit", "input_args": ["3 ETH"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "unknown"]}, {"description": "Stake stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "unknown", "unknown", "unknown"]}]}


In [11]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_usdt_for_usdc_uniswap", "description": "Swap 100 USDT for USDC on Uniswap", "total_steps": 2, "steps": [{"description": "Approve USDT for Uniswap", "to": "UniswapAddress", "value": "0", "function_name": "approve", "input_args": ["USDTAddress", "100"]}, {"description": "Swap USDT to USDC", "to": "UniswapAddress", "value": "unknown", "function_name": "swap", "input_args": ["USDTAddress", "USDCAddress", "100"]}]}


In [12]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_0.001_eth_for_usdc_on_uniswap", "description": "Swap 0.001 ETH for USDC on Uniswap", "total_steps": 1, "steps": [{"description": "Swap 0.001 ETH for USDC", "to": "UniswapAddress", "value": "0.001", "function_name": "swapExactETHForTokens", "input_args": ["0", "USDCAddress", "recipientAddress", "deadline"]}]}


In [13]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())

{"id": "yield_farming_multiple_protocols", "description": "Yield Farming with Multiple Protocols", "total_steps": 4, "steps": [{"description": "Deposit ETH into AAVE to earn interest.", "to": "0x893411580e590D62dDBca8a703d61Cc4A8c7b2b9", "value": "unknown", "function_name": "depositETH", "input_args": ["userAddress", "unknown", "0"]}, {"description": "Use aToken (aETH) as collateral to borrow DAI.", "to": "0xeA51d7853EEFb32b6ee06b1C12E6dcCA88Be0fFE", "value": "0", "function_name": "borrowDAITx", "input_args": ["aETHAmount", "userAddress"]}, {"description": "Deposit the borrowed DAI into Yearn Finance.", "to": "YearnFinanceAddress", "value": "unknown", "function_name": "depositDAITx", "input_args": ["DAIAmount", "userAddress"]}, {"description": "Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees.", "to": "UniswapPoolAddress", "value": "unknown", "function_name": "addLiquidityTx", "input_args": ["YearnVaultTokens", "userAddress"]}]}
